# Gibbs energy minimization (fixed T, P, bulk composition)
Closed system; crystallization of a rhyolitic liquid using rhyolite-MELTS

In [ ]:
import numpy as np
import scipy.optimize as opt
import scipy.linalg as lin 
import sys

In [ ]:
from thermoengine import core, phases, model, equilibrate
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

## Create phases for equilibrium assemblages

In [ ]:
src_obj = core.get_src_object('EquilibrateUsingMELTSv102')
modelDB = model.Database(liq_mod='v1.0')

In [ ]:
Liquid = modelDB.get_phase('Liq')
Feldspar = modelDB.get_phase('Fsp')
Quartz = modelDB.get_phase('Qz')
Spinel = modelDB.get_phase('SplS')
Opx = modelDB.get_phase('Opx')
RhomOx = modelDB.get_phase('Rhom')

The Berman model database provides the SWIM water model by default.  Instead, override that choice by instantiating the MELTS 1.0.2 water model directly.

In [ ]:
Water = phases.PurePhase('WaterMelts', 'H2O', calib=False)

## Define elements in system and phases in system

In [ ]:
elm_sys = ['H','O','Na','Mg','Al','Si','P','K','Ca','Ti','Cr','Mn','Fe','Co','Ni']
phs_sys = [Liquid, Feldspar, Water, Quartz, Spinel, Opx, RhomOx]

## Composition of the system
This is a high-silica rhyolite

In [ ]:
grm_oxides = {
    'SiO2':  77.5, 
    'TiO2':   0.08, 
    'Al2O3': 12.5, 
    'Fe2O3':  0.207,
    'Cr2O3':  0.0, 
    'FeO':    0.473, 
    'MnO':    0.0,
    'MgO':    0.03, 
    'NiO':    0.0, 
    'CoO':    0.0,
    'CaO':    0.43, 
    'Na2O':   3.98, 
    'K2O':    4.88, 
    'P2O5':   0.0, 
    'H2O':    5.5
}

Cast this composition as moles of elements for input to the Equilibrate class

In [ ]:
mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
moles_end,oxide_res = Liquid.calc_endmember_comp(
    mol_oxide_comp=mol_oxides, method='intrinsic', output_residual=True)
if not Liquid.test_endmember_comp(moles_end):
    print ("Calculated composition is infeasible!")
mol_elm = Liquid.covert_endmember_comp(moles_end,output='moles_elements')

In [ ]:
blk_cmp = []
for elm in elm_sys:
    index = core.chem.PERIODIC_ORDER.tolist().index(elm)
    blk_cmp.append(mol_elm[index])
blk_cmp = np.array(blk_cmp)

## Instantiate class instance and run calculation

In [ ]:
elmLiqMat = Liquid.props['element_comp']
idx = np.argwhere(np.all(elmLiqMat[..., :] == 0, axis=0))
conLiqMat = np.linalg.inv(np.delete(elmLiqMat, idx, axis=1))
conLiqMat[np.abs(conLiqMat) < np.finfo(np.float).eps] = 0
conPhs_d = {}
for phs in [Quartz, Feldspar, Spinel, Opx, RhomOx]:
    name = phs.props['phase_name']
    elmPhsMat = phs.props['element_comp']
    elmPhsMat = (np.delete(elmPhsMat, idx, axis=1))
    conPhsMat = np.matmul(elmPhsMat, conLiqMat)
    conPhsMat[np.abs(conPhsMat) < 100*np.finfo(np.float).eps] = 0
    conPhs_d[name] = conPhsMat

In [ ]:
for key,val in conPhs_d.items():
    for x in val:
        print ('{0:<15.15s}'.format(key), end=' = ')
        for indx,y in enumerate(x):
            if y != 0.0:
                print (' {0:+6.3f} {1:<10.10s}'.format(y, Liquid.props['endmember_name'][indx]), end=' ')
        print ()

In [ ]:
def saturation_curve(t, p, blk_cmp, NNO_offset, phs):
    global state, equil, conPhs_d
    if state is None:
        state = equil.execute(t, p, bulk_comp=blk_cmp, con_deltaNNO=NNO_offset, debug=0, stats=False)
    else:
        state = equil.execute(t, p, state=state, con_deltaNNO=NNO_offset, debug=0, stats=False)
    muLiq = state.dGdn(t=t, p=p, element_basis=False)[:,0]
    mLiq  = state.compositions(phase_name='Liquid')
    phase_name = phs.props['phase_name']
    assert phase_name in conPhs_d, phase_name+" is not in conPhs_d"
    muPhase = []
    conPhsMat = conPhs_d[phase_name]
    for i in range(0,conPhsMat.shape[0]):
        sum = 0.0
        for j in range (0,conPhsMat.shape[1]):
            if conPhsMat[i,j] != 0 and mLiq[j] == 0:
                sum = 0.0
                break
            sum += conPhsMat[i,j]*muLiq[j]
        muPhase.append(sum)
    muPhase = np.array(muPhase)
    result = phs.affinity_and_comp(t, p, muPhase, method='special')
    return result[0] 

In [ ]:
t = 1050
p = 1750
NNO_offset = 0.0
equil = equilibrate.Equilibrate(elm_sys, [Liquid, Water])
for ic,phs in enumerate([Quartz, Feldspar, Spinel, Opx, RhomOx]): 
    tk = t
    state = None
    result = opt.root_scalar(saturation_curve, 
                             bracket=(900,1100), 
                             x0=tk, x1=tk-25, 
                             xtol=.1, 
                             args=(p, blk_cmp, NNO_offset, phs), 
                             method='secant')
    tk = result.root
    print ('Liquidus for {0:<20.20s} is at {1:6.1f} °C at {2:8.1f} MPa'.format(phs.props['phase_name'], tk-273.15, p/10.))